
# **MIP4_Run3**

In [1]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 60.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 6.3 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [2]:

import openai
import os
from IPython.display import Markdown


### **Accessing the GPT4 API**

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']
openai.api_key = my_api_key


### **Generate Mathematical Model**

In [4]:
problem = """You are task with maximizing the water flow in a network of pipes over 10 time steps. The water enters the system at point A and exits the system at point G. Hence, you need to maximize the amount of water that flows to point G. The water enters point A according to the function f(t) = max(-4/30t^3+t^2-0.234t+3, 0), where t is the time step. There are serveral points that are connected through pipes. Each pipe has a fixed capacity. Each point has to possibility to release water out of the system in case the outgoing pipes are full. There are 12 available pipes in total but only 10 of them can be activated. Deactivated pipes can not be used to transport water. The activation status of a pipe is set before any water arrives at point A and does not change. Finally, water can not be stored at the connection points which means that any water coming in at time t needs to be flowing out at time t + 1.

These are the pipe capacities:
AB: 3
AC: 6
AF: 1
BC: 3
BD: 10
BE: 4
CE: 4
CF: 4
DE: 5
EG: 5
FB: 12
FG: 7"""

In [5]:

client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the variables for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 3
)


In [6]:
print(response1.choices[0].message.content)

1. Decision variables:
   - The flow of each pipe at time t, denoted by x_ij(t) for all i, j in [A,B,C,D,E,F,G] at t in [1,2,3,...,10]
   - The binary activation status variable s_ij for each pipe connecting points i and j G, i,j in [A,B,C,D,E,F,G].

2. Auxiliary Variables:
   - The amount of water entering from A, denoted by f(t) = max(-4/30t^3+t^2-0.234t+3, 0)
   - Pipe capacities denoted by C_ij, for all i, j in [A,B,C,D,E,F,G] where:
     - C_AB = 3
     - C_AC = 6
     - C_AF = 1
     - C_BC = 3
     - C_BD = 10
     - C_BE = 4
     - C_CE = 4
     - C_CF = 4
     - C_DE = 5
     - C_EG = 5
     - C_FB = 12
     - C_FG = 7
3. Other Variables:
   - The total amount of water that comes out of the system at G at the end of each time t, denoted by O_G(t) for t in [1,2,3,...,10].
   - The total amount of water flow at time t, denoted as T(t) for t in [1,2,...,10].


In [7]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the objective function for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 3
)

In [8]:
print(response2.choices[0].message.content)

The objective function is to maximize the total water flow out of the system at point G over 10 time steps. Therefore, it can be formulated as follows:

Maximize ∑ O_G(t) from t=1 to 10


In [9]:
response3 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the constraints for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content +  response2.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 3
)

In [10]:
print(response3.choices[0].message.content)

Constraints:
1. Flow conservation: The sum of the flow out of a node must be equal to the sum of the flow into the node plus any water released. For every node i in [A,B,C,D,E,F] and for every time step t in [1,2,...,10]:
    - For node A: f(t) = ∑ x_Aj(t)
    - For nodes [B,C,D,E,F]: ∑ x_ij(t-1) = ∑ x_ji(t) ∀ j≠i
    - For node G, to represent the total amount of water that comes out: O_G(t) = ∑ x_iG(t)

2. Pipe capacity: The flow in a pipe at any time cannot exceed its capacity. For every pipe (i, j), for every time step t in [1,2,...,10]:
    - x_ij(t) ≤ s_ij * C_ij ∀ i,j ∈[A,B,C,D,E,F,G]

3. Pipes' activation: The number of activated pipes cannot exceed 10.
    - ∑ s_ij ≤ 10 ∀ i,j ∈[A,B,C,D,E,F,G]

4. Non-negativity: The flow in a pipe at any time cannot be negative:
    - x_ij(t) ≥ 0 ∀ i, j ∈[A,B,C,D,E,F,G]

5. Binary nature of the pipe's activation status:
    - s_ij ∈ {0,1} ∀ i,j ∈[A,B,C,D,E,F,G]
  
6. The water released out from the system and the water flowing out at the end p

### **Generate Pyomo Code**

In [43]:
response4 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content + response2.choices[0].message.content + response3.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 3
)

In [44]:
Markdown(response4.choices[0].message.content)

Here is the Python Pyomo model for your problem.

```Python
from pyomo.environ import ConcreteModel, Set, Param, Var, NonNegativeReals, Binary, maximize, Objective, Constraint, ConstraintList, quicksum
import numpy as np

time_steps = [t for t in range(1, 11)] #Sample data for time steps
points = ['A', 'B', 'C', 'D', 'E', 'F', 'G'] #Sample data for points
C_AB, C_AC, C_AF, C_BC, C_BD, C_BE, C_CE, C_CF, C_DE, C_EG, C_FB, C_FG = 3, 6, 1, 3, 10, 4, 4, 4, 5, 5, 12, 7 #Sample data for pipe capacities

model = ConcreteModel()

# Set definition
model.T = Set(initialize=time_steps)
model.I = Set(initialize=points)
model.J = Set(initialize=points)

# Param definition (pipe capacities)
model.C = Param(model.I, model.J, initialize={('A', 'B'):C_AB, ('A', 'C'):C_AC, ('A', 'F'):C_AF, ('B', 'C'):C_BC, ('B', 'D'):C_BD, ('B', 'E'):C_BE, ('C', 'E'):C_CE, ('C', 'F'):C_CF, ('D', 'E'):C_DE, ('E', 'G'):C_EG, ('F', 'B'):C_FB, ('F', 'G'):C_FG})

# Variable definition 
model.x = Var(model.I, model.J, model.T, domain=NonNegativeReals)
model.s = Var(model.I, model.J, domain=Binary)
model.O = Var(model.T, domain=NonNegativeReals)

# Objective definition
model.obj = Objective(expr = sum(model.O[t] for t in model.T), sense=maximize)

# Constraints definition
model.constraints = ConstraintList()

# Flow conservation constraints
for t in model.T:
    model.constraints.add(model.x['A', 'B', t] + model.x['A', 'C', t] + model.x['A', 'F', t] == max(-4/30*t**3+t**2-0.234*t+3, 0))
    for i in points[1:-1]:
        model.constraints.add(sum(model.x[i, j, t-1] for j in model.J if j!=i) == sum(model.x[j, i, t] for j in model.I if j!=i))
    model.constraints.add(sum(model.x[j, 'G', t] for j in model.I if j!='G') == model.O[t])

# Pipe capacity constraints
for t in model.T:
    for (i, j) in model.C:
        model.constraints.add(model.x[i, j, t] <= model.s[i, j]*model.C[i, j])

# Limit on number of activated pipes
model.constraints.add(sum(model.s[i, j] for (i, j) in model.s) <= 10)

# No flow in the first time constraints
for (i, j) in model.C:
    model.constraints.add(model.x[i, j, 1] == 0)

# Method to solve the problem
from pyomo.opt import SolverFactory
opt = SolverFactory('cplex')
results = opt.solve(model, load_solutions=True)

# Print solutions
for v_name in model.component_objects(Var):
    for index in v_name:
        print("%s[%s] = %.2f" % (v_name, str(index), model[index].value))
```

- You can replace your data as needed, it is currently filled with sample data as I didn't have your original data.
- The model will calculate the water flow through each pipe at each time step, the binary activation status of each pipe, and the total amount of water that comes out of the system at the end of each time step to maximize the overall water output.
- The solution is printed out at the end of code.
- Please make sure that you have the 'cplex' solver installed, or change the solver according to the solvers available in your environment.

### **Run the code Generated by GPT4**

In [45]:
from pyomo.environ import ConcreteModel, Set, Param, Var, NonNegativeReals, Binary, maximize, Objective, Constraint, ConstraintList, quicksum
import numpy as np

time_steps = [t for t in range(1, 11)] #Sample data for time steps
points = ['A', 'B', 'C', 'D', 'E', 'F', 'G'] #Sample data for points
C_AB, C_AC, C_AF, C_BC, C_BD, C_BE, C_CE, C_CF, C_DE, C_EG, C_FB, C_FG = 3, 6, 1, 3, 10, 4, 4, 4, 5, 5, 12, 7 #Sample data for pipe capacities

model = ConcreteModel()

# Set definition
model.T = Set(initialize=time_steps)
model.I = Set(initialize=points)
model.J = Set(initialize=points)

# Param definition (pipe capacities)
model.C = Param(model.I, model.J, initialize={('A', 'B'):C_AB, ('A', 'C'):C_AC, ('A', 'F'):C_AF, ('B', 'C'):C_BC, ('B', 'D'):C_BD, ('B', 'E'):C_BE, ('C', 'E'):C_CE, ('C', 'F'):C_CF, ('D', 'E'):C_DE, ('E', 'G'):C_EG, ('F', 'B'):C_FB, ('F', 'G'):C_FG})

# Variable definition
model.x = Var(model.I, model.J, model.T, domain=NonNegativeReals)
model.s = Var(model.I, model.J, domain=Binary)
model.O = Var(model.T, domain=NonNegativeReals)

# Objective definition
model.obj = Objective(expr = sum(model.O[t] for t in model.T), sense=maximize)

# Constraints definition
model.constraints = ConstraintList()

# Flow conservation constraints
for t in model.T:
    model.constraints.add(model.x['A', 'B', t] + model.x['A', 'C', t] + model.x['A', 'F', t] == max(-4/30*t**3+t**2-0.234*t+3, 0))
    for i in points[1:-1]:
        model.constraints.add(sum(model.x[i, j, t-1] for j in model.J if j!=i) == sum(model.x[j, i, t] for j in model.I if j!=i))
    model.constraints.add(sum(model.x[j, 'G', t] for j in model.I if j!='G') == model.O[t])

# Pipe capacity constraints
for t in model.T:
    for (i, j) in model.C:
        model.constraints.add(model.x[i, j, t] <= model.s[i, j]*model.C[i, j])

# Limit on number of activated pipes
model.constraints.add(sum(model.s[i, j] for (i, j) in model.s) <= 10)

# No flow in the first time constraints
for (i, j) in model.C:
    model.constraints.add(model.x[i, j, 1] == 0)

# Method to solve the problem
from pyomo.opt import SolverFactory
opt = SolverFactory('cplex')
results = opt.solve(model, load_solutions=True)

# Print solutions
for v_name in model.component_objects(Var):
    for index in v_name:
        print("%s[%s] = %.2f" % (v_name, str(index), model[index].value))

KeyError: "Index '('B', 'A', 0)' is not valid for indexed component 'x'"

### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**